<div style="width: 30%; float: right; margin: 10px; margin-right: 5%;">
    <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/d3/FHNW_Logo.svg/2560px-FHNW_Logo.svg.png" width="500" style="float: left; filter: invert(50%);"/>
</div>

<h1 style="text-align: left; margin-top: 10px; float: left; width: 60%;">
    Soziale Netzwerke Analysieren Projekt:<br> Raumsonden & Himmelskörper
</h1>

<p style="clear: both; text-align: left;">
    Bearbeitet durch Flurina Riner, Patrick Schürmann, Si Ben Tran im HS 2023.<br>Bachelor of Science FHNW in Data Science.
</p>

# 1 Setup & Libraries

Hier in diesem Abschnitt importieren wir die wichtigsten Libraries für unser Projekt.

In [13]:
# Reload modules
%load_ext autoreload
%autoreload 2

# Standard library
from collections import defaultdict

# Data manipulation
import pandas as pd
import numpy as np
import json

# Visualisation
import matplotlib.pyplot as plt

# Network
import networkx as nx

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 2 Explorative Datenanalyse

Bewertungskriterien:   
Explorative Analysen der verwendeten Daten (Missing Values, Verteilungen, Plausilitätsprüfungen), allfällige Transformationen und Bereinigen oder Ausschliessen von fehlerhaften Daten. Wurde  geprüft, ob die Daten mit den Quelldaten übereinstimmen.

## 2.1 Daten einlesen

In [14]:
# Leere Liste für die Episoden
lst_every_episode = []
graph_every_episode = []

# Loop über alle Episoden
for episode in range(1, 8):
    filename = f'data/Star Wars/starwars-episode-{episode}-interactions-allCharacters.json'
    with open(filename, 'r') as file:
        data = json.load(file)

        nodes = data['nodes']

        df_nodes = pd.DataFrame(nodes)
        df_nodes = df_nodes.reset_index(drop=False)
        df_nodes = df_nodes.rename(columns={"index": "ID"})

        edges = data['links']
        df_edges = pd.DataFrame(edges)

        # Liste mit allen Episoden mit Nodes und Edges
        lst_every_episode.append((df_nodes, df_edges))

        # Graph Initialisieren
        G = nx.Graph()

        # Knoten und Kanten hinzufügen
        for idx, row in df_nodes.iterrows():
            G.add_node(row['ID'], label=row['name'], size=row['value'], color=row['colour'])

        for idx, row in df_edges.iterrows():
            G.add_edge(row['source'], row['target'], weight=row['value'])
        
        # Graph speichern
        graph_every_episode.append(G)

## 2.2 Fehlende Werte

## 2.3 Verteilungen der Daten

## 2.4 Plausibilitätsprüfungen